In [1]:
import tensorflow as tf
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import os
from tqdm import tqdm
import xml.etree.ElementTree as ET
import random
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report


In [2]:
import fonctions_from_notebook_1

In [3]:
images_dir_OG= "./Datas/Images"
annotations_dir_OG= "./Datas/Annotation"
images_dir_S= "./Datas/Images_Small"
annotations_dir_S= "./Datas/Annotation_Small"
images_dir_M= "./Datas/Images_M"
annotations_dir_M= "./Datas/Annotation_M"
output_size=(150, 150)
#output_size=(224, 224)
my_input_shape =(150,150,3)

In [4]:
datas_raw =fonctions_from_notebook_1.get_dogs_picture_breed_raw_pics(images_dir_M,annotations_dir_M,output_size)
datas_altered =fonctions_from_notebook_1.get_dogs_picture_breed(images_dir_M,annotations_dir_M,output_size)

Processing directories: 100%|██████████| 31/31 [00:53<00:00,  1.72s/it, current_directory=./Datas/Images_M\n02093428-American_Staffordshire_terrier]


In [5]:
print(datas_raw['breed'].unique())
my_num_classes = len(datas_raw['breed'].unique())

['Chihuahua' 'Japanese_spaniel' 'Maltese_dog' 'Pekinese' 'Shih-Tzu'
 'Blenheim_spaniel' 'papillon' 'toy_terrier' 'Rhodesian_ridgeback'
 'Afghan_hound' 'basset' 'beagle' 'bloodhound' 'bluetick'
 'black-and-tan_coonhound' 'Walker_hound' 'English_foxhound' 'redbone'
 'borzoi' 'Irish_wolfhound' 'Italian_greyhound' 'whippet' 'Ibizan_hound'
 'Norwegian_elkhound' 'otterhound' 'Saluki' 'Scottish_deerhound'
 'Weimaraner' 'Staffordshire_bullterrier' 'American_Staffordshire_terrier']


In [6]:
label_encoder = LabelEncoder()
datas_raw['breed'] = label_encoder.fit_transform(datas_raw['breed'])
datas_altered['breed'] = label_encoder.fit_transform(datas_raw['breed'])

In [7]:
datas_raw.info

<bound method DataFrame.info of                                                   image  breed
0     <PIL.Image.Image image mode=RGB size=150x150 a...      3
1     <PIL.Image.Image image mode=RGB size=150x150 a...      3
2     <PIL.Image.Image image mode=RGB size=150x150 a...      3
3     <PIL.Image.Image image mode=RGB size=150x150 a...      3
4     <PIL.Image.Image image mode=RGB size=150x150 a...      3
...                                                 ...    ...
5443  <PIL.Image.Image image mode=RGB size=150x150 a...      1
5444  <PIL.Image.Image image mode=RGB size=150x150 a...      1
5445  <PIL.Image.Image image mode=RGB size=150x150 a...      1
5446  <PIL.Image.Image image mode=RGB size=150x150 a...      1
5447  <PIL.Image.Image image mode=RGB size=150x150 a...      1

[5448 rows x 2 columns]>

In [8]:
train_val_df_raw, test_df_raw = train_test_split(datas_raw, test_size=0.2, stratify=datas_raw['breed'], random_state=42)
train_df_raw, val_df_raw = train_test_split(train_val_df_raw, test_size=0.25, stratify=train_val_df_raw['breed'], random_state=42)  # 0.25 x 0.8 = 0.2

# Separate features and labels
x_train_raw = np.stack(train_df_raw['image'].values)
y_train_raw = np.stack(train_df_raw['breed'].values)
x_val_raw = np.stack(val_df_raw['image'].values)
y_val_raw = np.stack(val_df_raw['breed'].values)
x_test_raw = np.stack(test_df_raw['image'].values)
y_test_raw = np.stack(test_df_raw['breed'].values)

# Convertir les labels en encodage one-hot
y_train_raw = to_categorical(y_train_raw, num_classes=my_num_classes)
y_val_raw = to_categorical(y_val_raw, num_classes=my_num_classes)
y_test_raw = to_categorical(y_test_raw, num_classes=my_num_classes)

In [9]:
train_val_df_altered, test_df_altered = train_test_split(datas_altered, test_size=0.2, stratify=datas_altered['breed'], random_state=42)
train_df_altered, val_df_altered = train_test_split(train_val_df_altered, test_size=0.25, stratify=train_val_df_altered['breed'], random_state=42)

# Separate features and labels
x_train_altered = np.stack(train_df_altered['image'].values)
y_train_altered = np.stack(train_df_altered['breed'].values)
x_val_altered = np.stack(val_df_altered['image'].values)
y_val_altered = np.stack(val_df_altered['breed'].values)
x_test_altered = np.stack(test_df_altered['image'].values)
y_test_altered = np.stack(test_df_altered['breed'].values)

# Convertir les labels en encodage one-hot
y_train_altered = to_categorical(y_train_altered, num_classes=my_num_classes)
y_val_altered = to_categorical(y_val_altered, num_classes=my_num_classes)
y_test_altered = to_categorical(y_test_altered, num_classes=my_num_classes)

In [10]:
# Création du modèle de réseau de neurones
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=my_input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(len(datas_raw['breed'].unique()), activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
history_raw = model.fit(
    x_train_raw,
    y_train_raw,
    steps_per_epoch=len(train_df_raw) // 32,
    epochs=30,
    validation_data=(x_val_raw, y_val_raw),# Ensemble de validation
    validation_steps=len(val_df_raw) // 32
)

Epoch 1/30
102/102 [==============================] - 289s 2s/step - loss: 6.4883 - accuracy: 0.0516 - val_loss: 3.3650 - val_accuracy: 0.0615
Epoch 2/30
102/102 [==============================] - 159s 2s/step - loss: 3.2645 - accuracy: 0.1002 - val_loss: 3.4046 - val_accuracy: 0.0734
Epoch 3/30
102/102 [==============================] - 122s 1s/step - loss: 3.0466 - accuracy: 0.1458 - val_loss: 3.5863 - val_accuracy: 0.0872
Epoch 4/30
102/102 [==============================] - 159s 2s/step - loss: 2.7546 - accuracy: 0.2418 - val_loss: 3.8204 - val_accuracy: 0.0890
Epoch 5/30
102/102 [==============================] - 179s 2s/step - loss: 2.3309 - accuracy: 0.3338 - val_loss: 4.0729 - val_accuracy: 0.0789
Epoch 6/30
102/102 [==============================] - 183s 2s/step - loss: 1.9586 - accuracy: 0.4460 - val_loss: 5.6234 - val_accuracy: 0.0752
Epoch 7/30
102/102 [==============================] - 150s 1s/step - loss: 1.4508 - accuracy: 0.5939 - val_loss: 6.4725 - val_accuracy: 0.0798

In [12]:
y_pred_raw = model.predict(x_test_raw)

35/35 [==============================] - 24s 436ms/step


In [13]:
print(y_test_raw)
print(y_pred_raw)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[4.8755123e-06 5.6741447e-07 4.4525743e-07 ... 5.6292229e-06
  3.0674223e-06 2.6777696e-09]
 [3.4615243e-07 1.4678483e-04 1.4876096e-03 ... 1.5989182e-09
  1.6763770e-04 6.6848169e-03]
 [3.4993958e-02 5.8181710e-03 4.0109383e-04 ... 7.1866743e-02
  4.6703381e-08 8.6096969e-07]
 ...
 [5.1631867e-15 6.4610565e-01 2.2173671e-14 ... 1.3062018e-13
  1.1395151e-21 1.5782537e-17]
 [3.9983189e-18 4.4523463e-07 4.7717213e-20 ... 1.8555973e-21
  6.9863385e-18 2.0294328e-14]
 [9.0099070e-03 4.3445162e-02 2.8395047e-04 ... 4.4530188e-03
  8.7477783e-05 5.5351364e-03]]


In [14]:
if len(y_pred_raw.shape) > 1 and y_pred_raw.shape[1] > 1:
    y_pred_raw = np.argmax(y_pred_raw, axis=1)
print(y_pred_raw)
if len(y_test_raw.shape) > 1 and y_test_raw.shape[1] > 1:
    y_test_raw = np.argmax(y_test_raw, axis=1)
print(y_test_raw)

[ 4 20 17 ...  1  5  6]
[ 0  6 13 ... 29 25 25]


In [15]:
accuracy_adam = accuracy_score(y_test_raw, y_pred_raw)
print(f"Accuracy: {accuracy_adam:.2f}")

recall = recall_score(y_test_raw, y_pred_raw, average='micro')
print(f"Recall: {recall:.2f}")

f1 = f1_score(y_test_raw, y_pred_raw, average='micro')
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.07
Recall: 0.07
F1 Score: 0.07


In [16]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_altered = model.fit(
    x_train_altered,
    y_train_altered,
    steps_per_epoch=len(train_df_altered) // 32,
    epochs=30,
    validation_data=(x_val_raw, y_val_altered),# Ensemble de validation
    validation_steps=len(val_df_altered) // 32,
    callbacks=[early_stopping]
)

Epoch 1/30
102/102 [==============================] - 47s 436ms/step - loss: 3.9375 - accuracy: 0.0405 - val_loss: 3.4054 - val_accuracy: 0.0450
Epoch 2/30
102/102 [==============================] - 18s 182ms/step - loss: 3.3950 - accuracy: 0.0408 - val_loss: 3.4068 - val_accuracy: 0.0450
Epoch 3/30
102/102 [==============================] - 22s 218ms/step - loss: 3.3903 - accuracy: 0.0426 - val_loss: 3.4053 - val_accuracy: 0.0450
Epoch 4/30
102/102 [==============================] - 47s 467ms/step - loss: 3.3892 - accuracy: 0.0480 - val_loss: 3.4063 - val_accuracy: 0.0404
Epoch 5/30
102/102 [==============================] - 56s 531ms/step - loss: 3.3901 - accuracy: 0.0423 - val_loss: 3.4074 - val_accuracy: 0.0459
Epoch 6/30
102/102 [==============================] - 90s 882ms/step - loss: 3.3892 - accuracy: 0.0411 - val_loss: 3.5400 - val_accuracy: 0.0477
Epoch 7/30
102/102 [==============================] - 94s 922ms/step - loss: 3.3872 - accuracy: 0.0450 - val_loss: 3.4497 - val_ac

In [17]:
y_pred_altered = model.predict(x_test_altered)

if len(y_pred_altered.shape) > 1 and y_pred_altered.shape[1] > 1:
    y_pred_altered = np.argmax(y_pred_altered, axis=1)
print(y_pred_raw)
if len(y_test_altered.shape) > 1 and y_test_altered.shape[1] > 1:
    y_test_altered = np.argmax(y_test_altered, axis=1)
print(y_test_altered)

35/35 [==============================] - 8s 173ms/step
[ 4 20 17 ...  1  5  6]
[ 0  6 13 ... 29 25 25]


In [18]:
# Calculer les métriques
accuracy_adam = accuracy_score(y_test_altered, y_pred_altered)
print(f"Accuracy: {accuracy_adam:.2f}")

recall = recall_score(y_test_altered, y_pred_altered, average='micro')
print(f"Recall: {recall:.2f}")

f1 = f1_score(y_test_altered, y_pred_altered, average='micro')
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.05
Recall: 0.05
F1 Score: 0.05


# LeNet-5

In [19]:
def create_lenet5(input_shape, num_classes):
    model = Sequential()    
    # C1 - Convolutional Layer
    model.add(Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=input_shape, padding='same'))    
    # S2 - Subsampling Layer (MaxPooling)
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))    
    # C3 - Convolutional Layer
    model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu', padding='valid'))    
    # S4 - Subsampling Layer (MaxPooling)
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))    
    # C5 - Fully Connected Convolutional Layer
    model.add(Flatten())
    model.add(Dense(units=120, activation='relu'))    
    # F6 - Fully Connected Layer
    model.add(Dense(units=84, activation='relu'))    
    # Output Layer
    model.add(Dense(units=num_classes, activation='softmax'))
    
    return model


model_lenet5 = create_lenet5(my_input_shape, my_num_classes)
model_lenet5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_lenet5.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 150, 150, 6)       456       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 75, 75, 6)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 71, 71, 16)        2416      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 35, 35, 16)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 19600)             0         
                                                                 
 dense_2 (Dense)             (None, 120)              

In [20]:
history_net5_raw = model_lenet5.fit(
    x_train_raw,
    y_train_raw,
    epochs=30,
    batch_size=32,
    validation_data=(x_val_raw, y_val_raw)
)


Epoch 1/30
103/103 [==============================] - 38s 336ms/step - loss: 14.7424 - accuracy: 0.0382 - val_loss: 3.4001 - val_accuracy: 0.0541
Epoch 2/30
103/103 [==============================] - 45s 437ms/step - loss: 3.2304 - accuracy: 0.1004 - val_loss: 3.4333 - val_accuracy: 0.0624
Epoch 3/30
103/103 [==============================] - 42s 408ms/step - loss: 2.8574 - accuracy: 0.2105 - val_loss: 3.6480 - val_accuracy: 0.0477
Epoch 4/30
103/103 [==============================] - 36s 354ms/step - loss: 2.3010 - accuracy: 0.3577 - val_loss: 4.2973 - val_accuracy: 0.0615
Epoch 5/30
103/103 [==============================] - 48s 470ms/step - loss: 1.6798 - accuracy: 0.5324 - val_loss: 4.7735 - val_accuracy: 0.0468
Epoch 6/30
103/103 [==============================] - 60s 583ms/step - loss: 1.1914 - accuracy: 0.6778 - val_loss: 5.8180 - val_accuracy: 0.0716
Epoch 7/30
103/103 [==============================] - 56s 543ms/step - loss: 0.8473 - accuracy: 0.7671 - val_loss: 6.9147 - val_a

In [ ]:
y_pred_raw = model_lenet5.predict(x_test_raw)

if len(y_pred_raw.shape) > 1 and y_pred_raw.shape[1] > 1:
    y_pred_raw = np.argmax(y_pred_raw, axis=1)
print(y_pred_raw)
if len(y_test_raw.shape) > 1 and y_test_raw.shape[1] > 1:
    y_test_raw = np.argmax(y_test_raw, axis=1)
print(y_test_raw)

# Calculer les métriques
accuracy_adam = accuracy_score(y_test_raw, y_pred_raw)
print(f"Accuracy: {accuracy_adam:.2f}")

recall = recall_score(y_test_raw, y_pred_raw, average='micro')
print(f"Recall: {recall:.2f}")

f1 = f1_score(y_test_raw, y_pred_raw, average='micro')
print(f"F1 Score: {f1:.2f}")

In [ ]:
history_net5_altered = model_lenet5.fit(
    x_train_altered,
    y_train_altered,
    epochs=30,
    batch_size=16,
    validation_data=(x_val_altered, y_val_altered),
    callbacks=[early_stopping]
)


: 

In [ ]:
y_pred_altered = model_lenet5.predict(x_test_altered)

if len(y_pred_altered.shape) > 1 and y_pred_altered.shape[1] > 1:
    y_pred_altered = np.argmax(y_pred_altered, axis=1)
print(y_pred_raw)
if len(y_test_altered.shape) > 1 and y_test_altered.shape[1] > 1:
    y_test_altered = np.argmax(y_test_altered, axis=1)
print(y_test_altered)

# Calculer les métriques
accuracy_adam = accuracy_score(y_test_altered, y_pred_altered)
print(f"Accuracy: {accuracy_adam:.2f}")

recall = recall_score(y_test_altered, y_pred_altered, average='micro')
print(f"Recall: {recall:.2f}")

f1 = f1_score(y_test_altered, y_pred_altered, average='micro')
print(f"F1 Score: {f1:.2f}")

35/35 [==============================] - 0s 4ms/step
[15  7 24 ...  8 14 15]
[ 0  6 13 ... 29 25 25]
Accuracy: 0.05
Recall: 0.05
F1 Score: 0.05


# personnal NN

In [ ]:
def create_NN(input_shape, num_classes):
    model = Sequential()    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='valid'))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='valid'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='valid'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=num_classes, activation='softmax'))
    
    return model

In [ ]:
output_size_240=(240, 240)
input_shape_240 =(240,240,3)

datas_altered_240 =fonctions_from_notebook_1.get_dogs_picture_breed_raw_pics(images_dir_OG,annotations_dir_OG,output_size_240)
#datas_altered_240 =fonctions_from_notebook_1.get_dogs_picture_breed(images_dir_M,annotations_dir_M,output_size_240)

label_encoder = LabelEncoder()
datas_altered_240['breed'] = label_encoder.fit_transform(datas_altered_240['breed'])
my_num_classes = len(datas_altered_240['breed'].unique())

Processing directories: 100%|██████████| 31/31 [00:46<00:00,  1.52s/it, current_directory=./Datas/Images_M\n02093428-American_Staffordshire_terrier]


In [ ]:
model_NN = create_NN((240,240,3), my_num_classes)
model_NN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_NN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 240, 240, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 238, 238, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 236, 236, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 234, 234, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 117, 117, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 117, 117, 64)      0         
                                                        

In [ ]:
train_val_df_altered_240, test_df_altered_240 = train_test_split(datas_altered_240, test_size=0.2, stratify=datas_altered_240['breed'], random_state=42)
train_df_altered_240, val_df_altered_240 = train_test_split(train_val_df_altered_240, test_size=0.25, stratify=train_val_df_altered_240['breed'], random_state=42)

In [ ]:
# Separate features and labels
x_train_altered_240 = np.stack(train_df_altered_240['image'].values)
y_train_altered_240 = np.stack(train_df_altered_240['breed'].values)
x_val_altered_240 = np.stack(val_df_altered_240['image'].values)
y_val_altered_240 = np.stack(val_df_altered_240['breed'].values)
x_test_altered_240 = np.stack(test_df_altered_240['image'].values)
y_test_altered_240 = np.stack(test_df_altered_240['breed'].values)

# Convertir les labels en encodage one-hot
y_train_altered_240 = to_categorical(y_train_altered_240, num_classes=my_num_classes)
y_val_altered_240 = to_categorical(y_val_altered_240, num_classes=my_num_classes)
y_test_altered_240 = to_categorical(y_test_altered_240, num_classes=my_num_classes)

In [ ]:
history_240 = model_NN.fit(
    x_train_altered_240,
    y_train_altered_240,
    epochs=20,
    batch_size=32,
    validation_data=(x_val_altered_240, y_val_altered_240)
)


Epoch 1/20
103/103 [==============================] - 32s 197ms/step - loss: 5.0792 - accuracy: 0.0398 - val_loss: 3.3986 - val_accuracy: 0.0459
Epoch 2/20
103/103 [==============================] - 18s 178ms/step - loss: 3.3980 - accuracy: 0.0456 - val_loss: 3.3964 - val_accuracy: 0.0459
Epoch 3/20
103/103 [==============================] - 18s 177ms/step - loss: 3.3965 - accuracy: 0.0438 - val_loss: 3.3948 - val_accuracy: 0.0459
Epoch 4/20
103/103 [==============================] - 18s 177ms/step - loss: 3.3952 - accuracy: 0.0462 - val_loss: 3.3935 - val_accuracy: 0.0459
Epoch 5/20
103/103 [==============================] - 18s 177ms/step - loss: 3.3939 - accuracy: 0.0462 - val_loss: 3.3924 - val_accuracy: 0.0459
Epoch 6/20
103/103 [==============================] - 18s 178ms/step - loss: 3.3927 - accuracy: 0.0465 - val_loss: 3.3916 - val_accuracy: 0.0459
Epoch 7/20
103/103 [==============================] - 18s 178ms/step - loss: 3.3927 - accuracy: 0.0459 - val_loss: 3.3911 - val_ac

KeyboardInterrupt: 

In [ ]:
y_pred_240 = model_NN.predict(x_test_altered_240)

if len(y_pred_240.shape) > 1 and y_pred_240.shape[1] > 1:
    y_pred_240 = np.argmax(y_pred_240, axis=1)
print(y_pred_240)
if len(y_test_altered_240.shape) > 1 and y_test_altered_240.shape[1] > 1:
    y_test_altered_240 = np.argmax(y_test_raw, axis=1)
print(y_test_altered_240)

# Calculer les métriques
accuracy_adam = accuracy_score(y_test_altered_240, y_pred_240)
print(f"Accuracy: {accuracy_adam:.2f}")

recall = recall_score(y_test_altered_240, y_pred_240, average='micro')
print(f"Recall: {recall:.2f}")

f1 = f1_score(y_test_altered_240, y_pred_240, average='micro')
print(f"F1 Score: {f1:.2f}")